<a href="https://colab.research.google.com/github/Sheraz67/RAG-PMC/blob/main/CLIP_photo_organizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
code = '''
import streamlit as st
import os
import numpy as np
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import pickle
from pillow_heif import register_heif_opener

# Register HEIF opener
register_heif_opener()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
st.write(f"Using device: {device}")

# Load CLIP model and processor
@st.cache_resource
def load_clip_model():
    model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336").to(device)
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")
    return model, processor

clip_model, clip_processor = load_clip_model()

# Cache file for embeddings
CACHE_FILE = "embeddings.pkl"

# Encode images
def encode_images(uploaded_files):
    paths, embeddings = [], []
    for file in uploaded_files:
        try:
            image = Image.open(file)
            image = image.convert("RGB")
            inputs = clip_processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                embedding = clip_model.get_image_features(**inputs)
            paths.append(file.name)
            embeddings.append(embedding[0].cpu().numpy())
            st.write(f"Encoded: {file.name}")
        except Exception as e:
            st.error(f"Error processing {file.name}: {e}")

    if paths:
        with open(CACHE_FILE, "wb") as f:
            pickle.dump((paths, embeddings), f)
        st.success("Images encoded and cached!")
    return paths, embeddings

# Process text query
def process_query(query):
    refined_query = f"a photo of {query}"
    inputs = clip_processor(text=refined_query, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        embedding = clip_model.get_text_features(**inputs)
    return embedding[0].cpu().numpy()

# Find matches
def find_matches(query_embedding, image_embeddings, image_paths, top_k=3):
    similarities = [
        np.dot(query_embedding, img_emb) /
        (np.linalg.norm(query_embedding) * np.linalg.norm(img_emb))
        for img_emb in image_embeddings
    ]
    sorted_pairs = sorted(zip(image_paths, similarities), key=lambda x: x[1], reverse=True)
    return sorted_pairs[:top_k]

# Streamlit app
st.title("Text-Driven Photo Organizer")
st.markdown("Upload your photos and search them with natural language!")

# File uploader
uploaded_files = st.file_uploader("Upload Photos", accept_multiple_files=True, type=["jpg", "jpeg", "png", "heic", "HEIC"])

# Process uploaded files
if uploaded_files:
    if os.path.exists(CACHE_FILE):
        cached_files = pickle.load(open(CACHE_FILE, "rb"))[0]
        if sorted([f.name for f in uploaded_files]) != sorted(cached_files):
            os.remove(CACHE_FILE)
            st.write("New files detected, re-encoding...")

    if not os.path.exists(CACHE_FILE):
        paths, embeddings = encode_images(uploaded_files)
    else:
        with open(CACHE_FILE, "rb") as f:
            paths, embeddings = pickle.load(f)
        st.write("Loaded cached embeddings.")

    query = st.text_input("Search your photos (e.g., 'sunset')", "")
    if query:
        query_embedding = process_query(query)
        matches = find_matches(query_embedding, embeddings, paths)

        st.subheader(f"Top {len(matches)} Matches for '{query}'")
        cols = st.columns(min(3, len(matches)))

        for i, (path, score) in enumerate(matches):
            col = cols[i % len(cols)]
            file_obj = next((f for f in uploaded_files if f.name == path), None)

            if file_obj:
                caption_text = path + " | Score: " + "{:.4f}".format(score)
                col.image(file_obj, caption=caption_text, width=200)
            else:
                st.error("File not found: " + path)

# Reset button
if st.button("Reset Cache"):
    if os.path.exists(CACHE_FILE):
        os.remove(CACHE_FILE)
        st.success("Cache cleared! Upload new photos to start fresh.")
'''

# Save the code to app.py
with open('app.py', 'w') as f:
    f.write(code)

print("✅ Code successfully saved to app.py")


✅ Code successfully saved to app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 --allow-invalid-certy


In [3]:
!curl https://loca.lt/mytunnelpassword


34.75.239.177